# sample table

In [15]:
import seaborn as sb

df = sb.load_dataset('tips')
conn = "sqlite:///example"  
df.to_sql(
            'tips',            
            conn,                
            if_exists='replace'
           )
print(df)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


# DENSE_RANK()

DENSE_RANK() is a window function in SQL that assigns a rank to each row within a partition of a result set. It is similar to the RANK() function, but DENSE_RANK() does not leave gaps in the ranking sequence when there are ties.

For example, if two rows are tied for rank 1, the next rank assigned will be 2, not 3. This function is useful for scenarios where you want to rank items without skipping numbers.



In [27]:
import pandas as pd
query_result = pd.read_sql("""SELECT total_bill, tip, sex, DENSE_RANK() OVER (ORDER BY total_bill DESC) AS Rank
            FROM (SELECT CAST(total_bill AS INTEGER) AS total_bill, tip, sex
            FROM tips)""", con=conn)
query_result.head()

,total_bill,tip,sex,Rank
0,50,10.00,Male,1
1,48,6.73,Male,2
2,48,5.00,Male,2
3,48,9.00,Male,2
4,45,3.50,Male,3


# RANK()
 
RANK() is a window function in SQL that assigns a unique rank to each row within a partition of a result set. Unlike DENSE_RANK(), the RANK() function leaves gaps in the ranking sequence when there are ties. 

For example, if two rows are tied for rank 1, the next rank assigned will be 3, not 2. This function is useful for scenarios where you want to rank items and acknowledge the gaps created by ties.


In [28]:
import pandas as pd
query_result = pd.read_sql("""SELECT total_bill, tip, sex, RANK() OVER (ORDER BY total_bill DESC) AS Rank
            FROM (SELECT CAST(total_bill AS INTEGER) AS total_bill, tip, sex
            FROM tips)""", con=conn)
query_result.head()

,total_bill,tip,sex,Rank
0,50,10.00,Male,1
1,48,6.73,Male,2
2,48,5.00,Male,2
3,48,9.00,Male,2
4,45,3.50,Male,5
